# 分词-LSTM+CRF
# Keras

In [2]:
# -*- coding: utf-8 -*-
train_data_txt = "../training/msr_training.txt"
init_train_data = []
with open(train_data_txt,encoding='gbk') as trt:
    init_train_data = trt.readlines()
all_data = "".join(init_train_data)
train_data_split = [line.strip().split() for line in init_train_data]

print(train_data_split[0:5])
print(len(train_data_split))

[['“', '人们', '常', '说', '生活', '是', '一', '部', '教科书', '，', '而', '血', '与', '火', '的', '战争', '更', '是', '不可多得', '的', '教科书', '，', '她', '确实', '是', '名副其实', '的', '‘', '我', '的', '大学', '’', '。'], ['“', '心', '静', '渐', '知', '春', '似', '海', '，', '花', '深', '每', '觉', '影', '生', '香', '。'], ['“', '吃', '屎', '的', '东西', '，', '连', '一', '捆', '麦', '也', '铡', '不', '动', '呀', '？'], ['他', '“', '严格要求', '自己', '，', '从', '一个', '科举', '出身', '的', '进士', '成为', '一个', '伟大', '的', '民主主义', '者', '，', '进而', '成为', '一', '位', '杰出', '的', '党外', '共产主义', '战士', '，', '献身', '于', '崇高', '的', '共产主义', '事业', '。'], ['“', '征', '而', '未', '用', '的', '耕地', '和', '有', '收益', '的', '土地', '，', '不准', '荒芜', '。']]
86924


In [3]:
# 读取字符词典文件
char_vocab_path = "../training/char_vocabs.txt"
special_words = ['<PAD>', '<UNK>'] # 特殊词表示
with open(char_vocab_path, "r", encoding="utf8") as fo:
    char_vocabs = [line.strip() for line in fo]
char_vocabs = special_words + char_vocabs
print(char_vocabs[0:10])

# 字符和索引编号对应
idx2vocab = {idx: char for idx, char in enumerate(char_vocabs)}
vocab2idx = {char: idx for idx, char in idx2vocab.items()}

['<PAD>', '<UNK>', '!', '#', '%', '&', "'", '(', ')', '*']


In [4]:
# "BMES"标记的标签
label2idx = {"S": 0,
             "B": 1, 
             "M": 2,
             "E": 3
             }
# 索引和BMES标签对应
idx2label = {idx: label for label, idx in label2idx.items()}

In [5]:
# 读取data
def data_list(path, vocab2idx, label2idx):
    with open(train_data_txt,encoding='gbk') as trt:
        lines = trt.readlines()
    datas, labels= [], []
    for line in lines: # line是一个字符串
        mline = line.strip().split() #mline是个词列表
        sent, tag = [],[]
        for i,word in enumerate(mline):
            if len(word)==1:
                sent.append(word)
                tag.append("S")
            else:
                s_word = ["M" for i in word]
                s_word[0] = "B"
                s_word[-1] = "E"
                sent+=list(word)
                tag+=s_word

        sent_ids = [vocab2idx[char] if char in vocab2idx else vocab2idx['<UNK>'] for char in sent]
        tag_ids = [label2idx[label] if label in label2idx else 0 for label in tag]
        datas.append(sent_ids)
        labels.append(tag_ids)

    return datas, labels

In [6]:
datas, labels = data_list(train_data_txt, vocab2idx, label2idx)
print(len(datas),len(labels))
print(datas[5])
print([idx2vocab[idx] for idx in datas[5]])
print(labels[5])
print([idx2label[idx] for idx in labels[5]])

86924 86924
[122, 5800, 6506, 563, 2572, 240, 3007, 2530, 6209, 6440, 6802, 3007, 2530, 3903, 2102, 2065, 6209, 6440, 182]
['“', '这', '首', '先', '是', '个', '民', '族', '问', '题', '，', '民', '族', '的', '感', '情', '问', '题', '。']
[0, 0, 1, 3, 0, 0, 1, 3, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0]
['S', 'S', 'B', 'E', 'S', 'S', 'B', 'E', 'B', 'E', 'S', 'B', 'E', 'S', 'B', 'E', 'B', 'E', 'S']


In [7]:
snum = 70000
train_datas = datas[0:snum]
train_labels = labels[0:snum]
test_datas = datas[snum:]
test_labels = labels[snum:]
print(train_labels[0:10])

[[0, 1, 3, 0, 0, 1, 3, 0, 0, 0, 1, 2, 3, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 1, 2, 2, 3, 0, 1, 2, 3, 0, 0, 1, 3, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 3, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 2, 2, 3, 1, 3, 0, 0, 1, 3, 1, 3, 1, 3, 0, 1, 3, 1, 3, 1, 3, 1, 3, 0, 1, 2, 2, 3, 0, 0, 1, 3, 1, 3, 0, 0, 1, 3, 0, 1, 3, 1, 2, 2, 3, 1, 3, 0, 1, 3, 0, 1, 3, 0, 1, 2, 2, 3, 1, 3, 0], [0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0], [0, 0, 1, 3, 0, 0, 1, 3, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0], [0, 0, 0, 0, 1, 3, 1, 2, 3, 0, 0, 1, 2, 3, 0, 0, 1, 3, 0], [0, 1, 3, 1, 3, 1, 3, 0, 1, 2, 3, 1, 3, 0, 1, 2, 3, 1, 2, 2, 3, 0, 1, 3, 0, 1, 3, 0], [0, 1, 3, 1, 3, 0, 0, 1, 3, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0, 0, 0, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0, 0], [0, 0, 1, 3, 0, 1, 3, 1, 3, 0, 1, 3, 0, 0, 0, 0]]


In [8]:
import numpy as np
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Masking, Embedding, Bidirectional, LSTM, Dense, Input, TimeDistributed, Activation
from keras.preprocessing import sequence
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras import backend as K
K.clear_session()

EPOCHS = 2
BATCH_SIZE = 64
EMBED_DIM = 128
HIDDEN_SIZE = 64
MAX_LEN = 100
VOCAB_SIZE = len(vocab2idx)
CLASS_NUMS = len(label2idx)
print(VOCAB_SIZE, CLASS_NUMS)

print('padding sequences')
train_datas = sequence.pad_sequences(train_datas, maxlen=MAX_LEN)
train_labels = sequence.pad_sequences(train_labels, maxlen=MAX_LEN)
test_datas = sequence.pad_sequences(test_datas, maxlen=MAX_LEN)
test_labels = sequence.pad_sequences(test_labels, maxlen=MAX_LEN)
print('x_train shape:', train_datas.shape)
print('x_test shape:', test_datas.shape)
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', test_labels.shape)

train_labels = keras.utils.to_categorical(train_labels, CLASS_NUMS)#.reshape((70000,100, 4))
test_labels = keras.utils.to_categorical(test_labels, CLASS_NUMS)#.reshape((16924,100, 4))
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', test_labels.shape)

## BiLSTM+CRF模型构建
inputs = Input(shape=(MAX_LEN,), dtype='int32')
x = Masking(mask_value=0)(inputs)
x = Embedding(VOCAB_SIZE, EMBED_DIM)(x)
x = Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True))(x)
x = TimeDistributed(Dense(CLASS_NUMS))(x)
outputs = CRF(CLASS_NUMS)(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

model.compile(loss=crf_loss, optimizer='adam', metrics=[crf_viterbi_accuracy])
model.fit(train_datas, train_labels, epochs=EPOCHS, verbose=1, validation_split=0.1)

score = model.evaluate(test_datas, test_labels, batch_size=BATCH_SIZE)
print(model.metrics_names)
print(score)

# save model
model.save_weights("../model/RNN_seg_weight.h5")
model.save("../model/RNN_seg.h5")

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

6874 4
padding sequences
x_train shape: (70000, 100)
x_test shape: (16924, 100)
trainlabels shape: (70000, 100)
testlabels shape: (16924, 100)


W0825 18:10:10.707118 140445235877696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0825 18:10:10.722083 140445235877696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



trainlabels shape: (70000, 100, 4)
testlabels shape: (16924, 100, 4)


W0825 18:10:11.936359 140445235877696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0825 18:10:12.069962 140445235877696 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:2403: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
masking_1 (Masking)          (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 128)          879872    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          98816     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 100, 4)            516       
_________________________________________________________________
crf_1 (CRF)                  (None, 100, 4)            44        
Total params: 979,248
Trainable params: 979,248
Non-trainable params: 0
_____________________________________________________

In [24]:
#类别向量定义
b = np.array([[0,1,2,3,4,5,6,7,8],
     [0,1,2,3,4,5,6,7,8]])
#调用to_categorical将b按照9个类别来进行转换
b = keras.utils.np_utils.to_categorical(b, 9).reshape((2,9,9))
print(b)


[[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1.]]

 [[1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1.]]]
